In [2]:
# Verify GPU
import tensorflow as tf 
tf.test.gpu_device_name()

'/device:GPU:0'

In [1]:
# Connect to images folder
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pickle
import os
def pickle_save(object, path):
  with open(path, "wb") as f:
    return pickle.dump(object, f)


def pickle_load(path):
  with open(path, "rb") as f:
    return pickle.load(f)


In [0]:
!ls "/content/drive/My Drive/Images"

ls: cannot access '/content/drive/My Drive/Images': No such file or directory


In [0]:
# Data processing
import pandas as pd
import cv2

IMAGES_DIR = "/content/drive/My Drive/Images/Images/"
DIR = "/content/drive/My Drive/Images/"
LABELS = 'https://raw.githubusercontent.com/ptran1203/facial_beauty/master/dataset/all_labels.csv'
def dataframe():
    df = pd.read_csv(LABELS)
    file_names = [IMAGES_DIR + file_name for \
                 file_name in df['filename']]
    df['score'] = [str(score) for \
                  score in df['score']]
    return df


def split(df ,rate):
    size = int(len(df) * rate)
    df1 = df.iloc[:size, :]
    df2 = df.iloc[size:, :]
    print(df1)
    return (df1, df2)

def dataset():
    images = []
    labels = []
    df = pd.read_csv(LABELS)
    for idx, row in df.iterrows():
      print("[process] ", idx)
      filepath = IMAGES_DIR + row['filename']
      label = row['score']
      image = cv2.imread(filepath)
      try:
        image = cv2.resize(image, (227, 227))
      except Exception as e:
        pass
      images.append(image)
      labels.append(label)
    return images, labels

imgs = pickle_load(DIR + 'images_arr.pkl')
labels = pickle_load(DIR + 'labels_arr.pkl')


In [5]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
import numpy as np

labels_np = np.array(labels)
imgs_np = np.array(imgs)



(trainX, testX, trainY, testY) = train_test_split(imgs_np,
	labels_np, test_size=0.25, random_state=42)



Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.models import model_from_json
from keras.optimizers import Adam

from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.core import Activation
from keras import backend as K
# from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator

#import data_processing as dp
import numpy as np

# Alexnet

class AlexNet:
    def __init__(self):
        model = Sequential()
        model.add(Conv2D(kernel_size=(11, 11),
                        activation='relu',
                        strides=(4, 4),
                        filters=96,
                        padding='valid',
                        input_shape=(227,227,3)))
        model.add(MaxPooling2D(pool_size=(2, 2),
                            strides=(2, 2),
                            padding='valid'))
        model.add(Conv2D(filters=256,
                        kernel_size=(11, 11),
                        strides=(1, 1),
                        padding='valid',
                        activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2),
                            strides=(2, 2),
                            padding='valid'))
        model.add(Conv2D(filters=384,
                        kernel_size=(3, 3),
                        strides=(1, 1),
                        padding='valid',
                        activation='relu'))
        model.add(Conv2D(filters=384,
                        kernel_size=(3, 3),
                        strides=(1, 1),
                        padding='valid',
                        activation='relu'))
        model.add(Conv2D(filters=256,
                        kernel_size=(3, 3),
                        strides=(1, 1),
                        padding='valid',
                        activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2),
                            strides=(2, 2),
                            padding='valid'))
        model.add(Flatten())
        # fully connected
        model.add(Dense(9216 ,input_shape=(224, 224, 3), activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(1000, activation='relu'))
        model.add(Dropout(0.4))
        # output
        model.add(Dense(1, activation='linear'))
        model.compile(loss='mean_absolute_error',
                      optimizer=Adam(lr=1e-4),
                      )
        self.model = model

    def model(self):
        return self.model


train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

# train_df, test_df = split(dataframe(), 0.7)

# train_generator = train_datagen.flow_from_dataframe(
#         dataframe=train_df,
#         x_col="filename",
#         y_col="score",
#         target_size=(227, 227),
#         class_mode='sparse')

# print(train_df)
# exit(0)
# validation_generator = test_datagen.flow_from_dataframe(
#         dataframe=test_df,
#         x_col="filename",
#         y_col="score",
#         target_size=(227, 227),
#         class_mode='sparse')

alax = AlexNet()
model = alax.model
H = model.fit_generator(
        train_datagen.flow(trainX, trainY, batch_size=200),
        steps_per_epoch=len(trainX) // 200,
        epochs=100,
        validation_data=test_datagen.flow(testX, testY),
        )



In [0]:

# Save model, ...
storage_path = "/content/drive/My Drive/Images"
pickle_save(labels_np, os.path.join(storage_path, "labels_np.pkl"))
pickle_save(imgs_np, os.path.join(storage_path, "images_np.pkl"))
model.save(storage_path + '/' + 'model.h5')
pickle_save(H, os.path.join(storage_path, "history.pkl"))


In [8]:
# Testing

imgpaths = [
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRw8S0D2cLWckXKsBKzOWNV-TNQA0RgFysLqAIopkosoIDNShSL&s',
  'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxITEhUTEhIWFhUWGRgVFxUWFRUVFRgWGBUXFxUXFRUYHSggGBolHRcVITEhJikrLi4uFx8zODMtNygtLisBCgoKDg0OGhAQFy0fHSUrLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tKy0rLS0tLS0tLf/AABEIARYAtgMBIgACEQEDEQH/xAAcAAABBAMBAAAAAAAAAAAAAAAAAgMEBQEGBwj/xABGEAABAwICBggDBgQEAwkAAAABAAIRAyEEMQUSQVFhcQYTIoGRobHwBzLBI0JictHhFCRSgjOSovGys+I0Q0RTY3ODk6P/xAAZAQEBAQEBAQAAAAAAAAAAAAAAAQQCAwX/xAApEQEBAAIBBAEDAgcAAAAAAAAAAQIRAxIhMUEEIjJRE3EUI2GBseHw/9oADAMBAAIRAxEAPwDuKEIQCEIQCEIQCEIQCEIQCEgVWxOsIymRmlawmJE7kGUIQgEIQgEIQgEIQgEIQgEIQgEIQgEIQgEIQgEIXMOnHxPFPWo4EF1QEtdUc3stIt2JzPE25oNp6Z9KmYSmQyrS6/NtN7tm8gZd8LjGlOnmknk62KIBBGqwNa2DsFr23rUNJ42u9xe90uJJLplxOZnaVAOIda5nkP0VVfHSVU2a9x2wCWn/ACzBUhunq7ACKj4Bt2iCOR2LWXvDthDt7RbvH6J2jiHixIPORI7/AKoOndEPivWou1MQTWpkQJP2jXCwMk9pu8d43LoFD4lYV9Voa4OpOgEi1Sm8/LrsOyxEibkb15vrQbgX3E3HIpLscbQSDIvtMZXGaD2S1wIkGQbgjKFlcI+FvxRLCzCYsktLgGVDk0RZvjHdK7rTeHAFpBBuCLgjeCohSEIQCEIQCEIQCEIQCEIQCEIQCw5wAk2A2rK0n4raf/hsJqsqalSqdUAHtFsHWO8C4Ejeg1H4j/EEvJw2FqQz7z2yHOidYAz8vGy5i6CL6zgeNj9ISRTcSSTe0TE8CRs5IrvtA7ROeezZMjwuqpl7miezl90nvsJgeqrX4ppmwbwLb9xJupFeu4N7UH8IOrYcs1CqP1iIpkcST5bUDRqOnOB72JzXtdwKZqQLe/BIbxQPt4Eg87Jtz73EHgfosat/rPoner4g94jwRGNW8i3GPouvfCr4ivpOZhsW8GiSWseT2mvcZa3iCQ4cJG+3IHtIuQY3AiD37k9hi2QHiJLXN+nPNUez2uBAIMg3BGRCyqforpWniMNTex0w0NdnZwABBkBXC5AhCEAhCEAhCEAhCEAhCEAvOnxB0hVxOPqB7w4UnGmwt+UAEkaovxPPuXftN4rqsPWqTGqxxB46pjzhea8QQC6/aNuUmXHibeSCv6to5X23dNiSfMnkihSJtAyMk2AbOQGwW74kzmJPWjhMRyk5cT+g3rMsAgnMgfrHABVVS8S77Ma141iLbre98ZJmsHXsByHjeblbFSwxcYaInuMbZjIcOW9W2B6IhxBqCR/T+v6LjLOYu8cLl4c5dgiRInwt3FI6hzcxHcusY3ow3MgmYa1gsJyAn6LGF6IU2CXN1jmJGZ/T/fcvG871/h3LqWHME6rrCSYmBtnKAg0rA3HGPYC6aejYJuLDtOn7x2DcBw8TMgMYzQbSHdnMg5cNnl4KfrrfjucvbAh5keDvFRK+ILnE+EfTct103ofUYDqzaMvGAtLrsg+8l74Z9Twyw6a3/wCEnTU4XEMo1ahbQcdU5EDdnkJMmLr0vSqBwDmkEESCDII3gjNeK8DWNN7Xi+qQYOVuC9k6FxJq4ejUIAL6bHEC4Bc0G0ALuuE1CEKAQhCAQhCAQhCAQhCCBp+m12GrBwkdW+0Fxs0kQBmZheVa1Uz4kxMQPlA3ZR3letMVRD2OYcnNLTssRGa8x6X0UaNWrRc3tNJabzlFp22jxQUtGqQIiXOLYnmD6wrfRej3VKro+VhcBx1LAxuLv+Ep/QOgX1ajHR2A6Cdlmy3zaB3redE6FFMOJ+84k8bmPKFm5eXXaNXDw9Xeq/o9obVgm5Nyc5AyHLatwoYYBScDg2gWUw0As27fLTrGdogMwsmSMhAHqT6eKU+gFNFND6SopK2G2bNvFRKuFBPvZ7HgrquBCrK2IaMyB6Kaq7jXukWDDmG2Xuy5hpXBQSV2iu1rhsIWldKtEhokCx9wtHDlq6ZefHt1RzcUDOQjjbzGS9g9G2BuEw4AgClTgREdgbCvKuBwuvXayw1nBs7LuiSvW+FohjGsGTWho7gAtTKdQhCAQhCAQhCAQhCAQhCAXG/i7ofq8S2uMqzSDwcLGN0ggzvXZFpPTkMxROEdaBrB9rVCDq33KZZTGbrvDC53Uaz0SofytMjaCfMx9FH0nhMTryztHZmADwnLxVv0ToxhqTdobHmU3p7EYhsCg0De9wkDuFzytzCxZXWbdjjvGRSDGaWpmeqDm8w71uB3K+0Z0iqFp6+nqkch9VremMRpGnU1aT31WuaHMeGNLNb7zXtAlozvrbs7kX+jNdx1XwSL6wa5oO/MLrLcm+znDGb13bFRxYcJCiY/SBbYDvUuhRAbkqvEXfGXFefXXp0Sta0jgcdiH9l4jiS1s8gJ95Jqn0ZxI+eo08tb1sVaaTw2IdSc6k4h+sIpRBNMggyTYPuHCbW25KowuArilJqVOvLiQHQ5rWQIa+BBM612kZjkvaW9O9vG4S5eFzo7R7qYIJn3ulVvSun9g8xlB8/3V9o4vLR1gvzn1VZ0ppzh6v5CfASueO7yjrkx1jWnfDDBMraSY1wcQJcIiQQJvaIMXXpFcV6KaNbQpU3stVqAVNYWMkawEjgfJde0Ri+tosftIvzFj6LVjnMmXPiuMl/KYhCF08ghCEAhCEAhCEAhCEAtFxTf5qq5+QPkCY8yt6WmadpEVKhAk6wdq/1AXgeK8eb7Wr4l+uz+is6K3w9P8oV2aAKouiP/AGdg2tLmnm17mnzC2Sms2U+p7zwhu0e071IpYVrRACeJhNOqJezrvWKgsqSs3tSrx7TCpsSwzYLi4u8KlUWSLpT8MEaOdrN5WI2qU5q7m9OL2qvqUgFr3SP/AAan5Hf8JWy1gtY6W2oVPyld8U+pxy36Roep/LUD95rGEA5EBgt4LoXQ1kYVnGfK30XOMBdlMgFrAwas7TqgeC6d0ZoFmGpA7i7ucS4eRC9eP7q8/kX+XItEIQvdjCEIQCEIQCEIQCEIQCqdOYYdl+2YP098VbJNRgcIIkHYplNzTrHLpu2h4XDdU54GTnvqD+9xefMuVmxye07gWsLHNBEyDJJyuM+9RWFYuTHprZx59UPyouPqVWj7KJ3kaw7wCD5p/WhIq1mgS5wA4lcyvWXuaGKqBoLm32xMSq3E4urrCA0N2gtLnO5EOGr4FWLcZTPyvbAzuB6qHiazHOs5p5EFd7dSXfg5gqsOJiAdin9aqtlk+youdplq05VK17T2EdWaaTQSXwABmTIj0V5UcldHaOtimn+gF3lA8yF68U7s3LexjRugajnMY5p1RGsSCAAInP0W/tEWCyhaMcZiz553LyEIQunAQhCAQhCAQhCAQhCAQhCCv07Q1qJjNvaHdn5StcpvstzIWoaWwnU1PwOkt+rTyXjzY7m3rxZaumHtJFlQYrRD9fWdWeRuIaY5WV5RepDqWsFlnZuwz6a1t+iTsqzxLBKhYrAPbk8H+391sj9HumxSW4CM13/Zo/X9KXQ+FrAy+qSNjYEeOauCE5qRko1Z8Lnyz55bu2az4Cu+huFID6p+8dVvIZ+fotUxNfYM1v8A0eZGGpD8IPjc+q0cU9sfLVihCF7vAIQhAIQhAIQhAIQhAIQhAIQhAKh6TAEsadzj6K+Wt6RrtqVHFsw3sTsJBJMbxeO5TKbl0uOWrLVGXupm9x5hTaGLadqzVpyq9+EEmLHgsFvqt8x33i1OJG9M1cRxVacJU2VPL902cHU2v8ldmkqtiQLkqrq4kuy8UqphYzM80qlTTejp2bFKBJXRNAk9RTBsQNX/ACkj6LTsFg9d0u+QZ8fwhbdoevOs05zI78/fFaeHerWbnslkiyQhC9ngEIQgEIQgEIQgEIQgEITBxIyYNY8MhzdkgfUSvjgPlGsf9Pjt7kp1Em7zP4RZvfv70w1us47m2Vc2q/H16rmkkw3c2wPfmVHptsFaY2mSznZVmFyg5jsnmLfv3qoWcPKh4rCuF9m8fVXlFiksoBeXJxY5/u9eLmyw8eGoAkLFR4K2PH6HDrssd2w8txVHUw7stUzkIvPKM1jywyxbseTHPvFXVCmaO0cXdp0huze7luHFXejuj4+arc7GZgfm38suatH4de3Hwb75PHl+RrtiqOpgRFhkNgTb3FoJGYurKsxVuJE9kZuMD1PkCtemLe1lhMa+BrXGwqwp4hp4e96iUG9gJQGq+Dk72U0u09CZ6uPlMengjrSPmHeFyuzyElrwcilIoQhCAJUV2LmzBrccmjv29yjOl57Zn8P3Rns25HNSGDwHsK6c7J6rW+d0/hFm/v3qSwCLCAmaTbX2p8iyUMYytqsJWaFLVpxtiTzOaaxA1nMbx1jyGXnCk1ckDNQS0BV2KoalUnY+D/cBHmIVkBZYxOH6xhGR2HcdhVDeHCmNUDA1NZt7OB1XDc4Z/r3qa1K5jXun/SMYHCOqBwFR56ulMfOZMwc9Voc7uA2rkGG6a4zWB/iHwHa0TDCZm4bFuAhbT8W8fjWVG1qFYto0h1b2NiZc69QgzrCdVsRaNsmNTw2LoYhk4hjWuP8A4iixrHtO+tSbDKg32a6MitPFj9O9bLZ4d06OaXZisPTrstrjtD+l4MPb3EFWLlzr4U4HF4Zj21zT6qq5xpBlzrNOr1hcBBa8ARN4a3euhvWayS9nVQ8QFEwdDWqF2xjSP7jn5BScdU1W2u4kNaN7jl+vcpGGw/VsDcztO87SlSBghsLOLp6zLZjtDmFkiydpGyim8NV1mgpx2SjYYapczcZHI/vKlAWQR3NE2sUptcjO43oqttySXDb79/oqiSxwNwhQCYOcTOVsj+4Qppepmk2CBtN/0UiLQo1d0azt0eE3UsjJVGK1m8k5Nk1i3Qx3IrFV0U+MQOZUUnCCS5++w5BO1Cs0WarQNwSCUGWhKplYprGRQQcZ9nVD/uv7LuBHyuVgxIxdAPaWptji1waflItzCCj6UYKnUEPYCDLXCM2kQQVxTQuhdWvQwz6jSyrVaD2iSKLqsUw50Wc9kOAGx7SYm3bel1UMaXO+UNc48g0k+S4R0frudVo1ahua1Ko8/wDytc6NwA8AANy08Et3py9GNpt1wIENgNGQbGUBTHquwLiazwdhUuq8ucWDY2SdxOUeazOkbDfaVi77tOzeLj8x+in1CmsDh+rYG95Szmgw4JdMrFRJCBGLEFr91jyKkTaeCTVZrNI3hN0XSziBB5hAqjdvNACTg3SxvJOAXREWo1pJByzQm6TpaHb59ShUJ0ranU/KVPpGQDwHoq7Tropu4hWNIdkch6KKjaSd9meMDzTrO1q8PVR9ImwG8hScMLKh55smUuqUhIF0lioFmks1MlApiZxtOW2zHaHcnKJslPNk9r6c6+Lmkg3BOg3qN6oc32Pg3WPcuVU6YbSjbqxHoPf6LavjDi9arQojIB1Q8y4Nb6PWpufrcgLfqvofHx1hv8vO97p3boVpYYnDU8QT2i3Uqf8AuMsT3iHf3K/wLbFxzcZ7ti5j8Hq7tXEUD8sNqDgZLSO8R/lXVKWQ5LHy49OVjuXbLk2wJdXJJpZLzBVSE5VyTSsD1M2TTxqyRt9UukVivkgjaMd2I3E+sqYVAwBjWHFT1BV6MM0mcvqhJ0IOxq/0kjzQqg0+fswN5hWoyVRp/OmN7h9FaudDZQQcQZeBuupmHKh0BMu3+iewb7lA/UN0hLqi6QinaKVUySMMl1MlPa+iaKb0hV1abjwS6SrukdSKcbzCe0ef+neN6zHP/DFMcmjtf6nOUeg249+4/RQazzUxNV2etUeZ2Aa5K2LQ+j9d4bOo0Ave85U6bbveeQnmSBtC+rJJjPxI87dL/objP4fE4QG38RVdI/8ASDH0aU86j6n/ANa7LhXSDwMLzvpDSJdiW12NLQxzDTZtbTpEdW2d9hO8knau+6Nrhz3xkTrDkbhYeeeL+XpOydVyRSyWKyzSXh6X2KuSZCer5JkZJAphulYg2SaYum8c+yqIzTD+asWmygVmSJGy6lYV8tBRFfoYw+qNzj6lCb0e7+YrDv8AT9UIEaafPUHe4eNj+vgrLFv7Hcqiu+aVGdlRvkCrJ5mFQ7TbDVHwr+0VKOSr6Lu0oLd10yUtjrJt5RTuGTrkxhk+FA0zNUXTKrq0weZ8BKvdq1b4ivjDuO5lQ/6CrpHCNC0/vHn9f38FsumanUU/4Zv+I6H4g/iF6dDkz5nfiIH3EzoUCgw4lwBLTqUGm4fWAkvI2spghx3uLRtKrbucSSSSSSTckkySTt2k/uvrWbup4n+f9OJ+ada0RJ9+x6rtnQ7E69HDP/roMn8zRqu8wVxPEmAGj37K6h8L8VrYei3/AMs1Wf8A6F48nhZvk4/SuLf6qcZkmnpxYXZOIyTDU7XyTDCgksUHHvUubKtxrrqiZT+VJwRiW8SlUD2U2bOlEQ8C2cVV/KCe+I9CsrOCf9vU4tb5EoVRVMqyGN3PnyKu6CoMqkcVfUcki1IJsq0mHqdrKvxdngqEWzTZYeU3SfZZcUD+GUgKNQT6KQ5an0/YH0iwvDAWuBe75WgiC48AJPctscVoPxRq/wAvV/J6uaFcfMHLtJ48Vqg6sFtKmOrpMObaYJOs78biS9x3uO4IZDRPv/f3sUXBi0+59+7IfVk8PXivr4zpjzve6PAzc5+/JdL+EzezVH9Lwf8AM3/p9VzXDt2+/fvauofDQNpmpTP+K6nTrPGxrSSKTI/q1Xax4PaNhWf5H2OvboKe2JkFOjJfOro3XyUZhUmsoqoeBVXi3dqFYhVeIHbRFlTNkioVlgskvViKl9bUqE72x5oTOkB2u5ClVHcfte9X1I2QhWFKBUTHCyEJQ7h32T+shCKk4R0qSsoUQ24LnPxKM0ao4M/5rEIXfH98/crlVZ0CB7CTTEnw9QPqFlC+rfuSfavtC0WjrKrxrsoM60sP/eO12sY0/hL3tLvwggZq/wDhlialTH1S52s+rSe5zjtPWU3E8OSELx5u8y/70uHh18UyBdOShC+apFVRQUIRC5zVVWMvHNCEFg0rD0IVg1/S1cMcCeVkIQiv/9k=',
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQM1KzeoH8xv7U_-ZE_-LNGEnSNTuudDNrIPh-YmGFrqYmiNoRnOA&s',
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS9Xo66Yk4iO_Yn_HJp2vMe1B4YDcHHdbiV1XUXnJtmyWJO2RvhwQ&s',
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTGL1JKRw4b3nneDMJepKhDAkB2F7bX270Fjnl50x4DTmZbEECyqA&s'
]
import urllib.request
from PIL import Image

imgs_input = []
for path in imgpaths:
  req = urllib.request.urlopen(path)
  arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
  img = cv2.imdecode(arr, -1)
  img = cv2.resize(img, (227, 227)) / 255.0
  img = np.expand_dims(img, axis=0)
  p = model.predict(img)[0]



[[3.0419333]]
[[2.885999]]
[[2.7845423]]
[[3.1951225]]
[[4.008596]]
